# Necessary libraries and credentials

In [1]:
import os 
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
import json
import boto3
import pandas as pd
from sqlalchemy import create_engine, text, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

import matplotlib.pyplot as plt
import plotly.express as px

import psycopg2

In [2]:
# s3 credentials
elo_s3_accessKeys = pd.read_csv("C:/Users/elodi/Documents/Documents/PYTHON/Jedha/data_science_full_stack/elo_s3_accessKeys.csv",encoding='utf-8-sig')
ACCESS_KEY_ID = elo_s3_accessKeys['Access key ID'].values[0]
SECRET_ACCESS_KEY = elo_s3_accessKeys['Secret access key'].values[0]
bucket_name = "bucket-getaround"

In [16]:
# RDS database credentials
elo_rds_accessKeys = pd.read_csv("C:/Users/elodi/Documents/Documents/PYTHON/Jedha/data_science_full_stack/elo_rds_accessKeys.csv",encoding='utf-8-sig')
db_host = elo_rds_accessKeys['db_host'].values[0]
db_port = elo_rds_accessKeys['db_port'].values[0]
db_name = elo_rds_accessKeys['db_name'].values[0]
db_user = elo_rds_accessKeys['db_user'].values[0]
db_pass = elo_rds_accessKeys['db_pass'].values[0]

# Scraping Booking.com

In [2]:
class booking_spider(scrapy.Spider):

    name = "booking"
    start_urls = ['https://www.booking.com']
    best_cities = ['Saintes Maries de la mer', 'Biarritz', 'Bayonne', 'Collioure', 'Aigues Mortes']

    # Retrieve the page for each CITY
    def start_requests(self):
        checkin = "2023-07-07"
        checkout = "2023-07-12"
        # warning: url from GoogleChrome (doesnt work for Edge)
        # city_url_template = 'https://www.booking.com/searchresults.fr.html?ss={}&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQm4ARfIAQzYAQHoAQGIAgGoAgO4AsHWlqIGwAIB0gIkZTU5NjhmNWEtMTk1Yi00OGEyLThmNWYtYzU2ZDNmMDU5MWZm2AIF4AIB&sid=4044a316139dbb224d418fbb7843881c&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_type=city&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure'
        city_url_template = 'https://www.booking.com/searchresults.fr.html?ss={}&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQm4ARfIAQzYAQHoAQGIAgGoAgO4AsHWlqIGwAIB0gIkZTU5NjhmNWEtMTk1Yi00OGEyLThmNWYtYzU2ZDNmMDU5MWZm2AIF4AIB&sid=4044a316139dbb224d418fbb7843881c&aid=304142&lang=fr&sb=1&src_elem=sb&src=index&dest_type=city&checkin={}&checkout={}&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
        
        for city in self.best_cities:
            city_url = city_url_template.format(city,checkin,checkout)
            yield scrapy.Request(city_url, headers=headers, callback=self.parse_hotels_with_urls, meta={'city':city, 'city_url': city_url})
    
    # Retrieve every HOTEL URL for a city
    def parse_hotels_with_urls(self, response):
        # Box corresponding to the hotel
        hotels = response.xpath("//h3[@class='a4225678b2']")
        # Inside this box, retrieve name and url
        for hotel in hotels:
            hotel_name = hotel.xpath(".//div[contains(@class, 'fcab3ed991 a23c043802')]/text()").get()
            hotel_url = hotel.xpath(".//@href").get()
            yield response.follow(hotel_url, callback=self.parse_hotels_details, meta={'city': response.meta['city'], 'hotel_name': hotel_name, 'hotel_url': hotel_url})
    
        # Loop over pages to retrive more hotels (offset set to 25 hotels max per page)
        hotel_ou_commencer = 25
        while hotel_ou_commencer < 1000:
            city_next_page_url = response.meta['city_url'] + f"&offset={hotel_ou_commencer}"
            yield response.follow(city_next_page_url, callback=self.parse_hotels_with_urls, meta=response.meta)
            hotel_ou_commencer += 25

    # Retrieve HOTEL INFORMATION from each hotel page
    def parse_hotels_details(self, response):
         yield {
              'city':response.meta['city'],
              'hotel_name': response.meta['hotel_name'],
              'hotel_url': response.meta['hotel_url'],
              'hotel_GPS': response.xpath("//@data-atlas-latlng").get(),
              'hotel_score': response.xpath("//div[@class='b5cd09854e d10a6220b4']/text()").get(),
              'hotel_voters':response.xpath("//div[@class='d8eab2cf7f c90c0a70d3 db63693c62']/text()").get(),
              'hotel_description': Selector(response).css('div.hp_desc_main_content').xpath('string()').get().strip()

         }
         
filename = "booking_scrap.json"

# If file already exists, remove it before
if filename in os.listdir('scrap_results/'):
        os.remove('scrap_results/' + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'scrap_results/' + filename : {"format": "json"},
    }
})

process.crawl(booking_spider)
process.start()

2023-07-08 17:01:09 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2023-07-08 17:01:09 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 37.0.1, Platform Windows-10-10.0.22621-SP0
2023-07-08 17:01:09 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2023-07-08 17:01:09 [scrapy.extensions.telnet] INFO: Telnet Password: c2461cd156879d51
2023-07-08 17:01:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2023-07-08 17:01:09 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermi

# Send to s3

In [4]:
# Merge API scraping of the weather (only the top5 cities) and hotels for those cities
df_booking_scrap = pd.read_json('scrap_results/booking_scrap.json')
df_top5cities_weather = pd.read_csv('scrap_results/forecast_weather_top5cities.csv')
df_merged = pd.merge(df_booking_scrap, df_top5cities_weather, on='city', how='left')

# Send to s3
df_merged.to_csv('scrap_results/forecast_weather_top5cities_and_hotels.csv', index=False)
df_merged.head()

,city,hotel_name,hotel_url,hotel_GPS,hotel_score,hotel_voters,hotel_description,feels_like,humidity,wind_speed,city_id,main_weather,main_weather_scores,final_score,lon,lat
0,Aigues Mortes,La Maison de Lyna,https://www.booking.com/hotel/fr/la-maison-de-...,"43.56601750,4.19235320","9,1",137 expériences vécues,Vous pouvez bénéficier d'une réduction Genius ...,26.37425,67.625,2.66575,1.0,clear sky,5.0,616.0,4.191284,43.565823
1,Saintes Maries de la mer,Les Rizières,https://www.booking.com/hotel/fr/les-rizieres....,"43.47103999,4.41058695","8,2",797 expériences vécues,Vous pouvez bénéficier d'une réduction Genius ...,25.21725,74.825,2.83075,29.0,clear sky,5.0,644.0,4.428717,43.452277
2,Saintes Maries de la mer,Thalacap Camargue,https://www.booking.com/hotel/fr/thalacap-cama...,"43.45349134,4.43545103","7,7",972 expériences vécues,Le Thalacap Camargue est situé en face de la p...,25.21725,74.825,2.83075,29.0,clear sky,5.0,644.0,4.428717,43.452277
3,Saintes Maries de la mer,L'auberge Camarguaise,https://www.booking.com/hotel/fr/lauberge-cama...,"43.47184235,4.41156596","8,6",377 expériences vécues,Dotée d'une piscine extérieure ouverte en sais...,25.21725,74.825,2.83075,29.0,clear sky,5.0,644.0,4.428717,43.452277
4,Biarritz,Apartment Les Hauts de Biarritz-4 by Interhome,https://www.booking.com/hotel/fr/apartment-les...,"43.47017600,-1.56181800","6,9",9 expériences vécues,"Situé à 1,6 km de la plage de Marbella, L'Apar...",21.36800,79.950,2.57900,9.0,clear sky,5.0,636.0,-1.559278,43.483252


In [5]:
# Create an instance of boto3.Session that connects with my aws account
session = boto3.Session(aws_access_key_id=ACCESS_KEY_ID, aws_secret_access_key=SECRET_ACCESS_KEY)

# Create a variable s3 that connects my session to the s3 ressource
s3 = session.resource("s3")

# Create a variable that will connect to my existing bucket bucket-getaround
bucket_getaround = s3.Bucket(bucket_name) 

# Upload file to the s3 bucket
bucket_getaround.upload_file("scrap_results/forecast_weather_top5cities_and_hotels.csv","forecast_weather_top5cities_and_hotels.csv")

# ETL

### Extract

In [6]:
# Connect to my s3
s3_client = boto3.client('s3' , aws_access_key_id=ACCESS_KEY_ID , aws_secret_access_key=SECRET_ACCESS_KEY)

# Retrieve file from s3
s3_file_path = 'forecast_weather_top5cities_and_hotels.csv'
local_destination = 'data_from_s3/forecast_weather_top5cities_and_hotels.csv'
s3_client.download_file(bucket_name, s3_file_path, local_destination)

### Transform

In [7]:
# Clean data (drop lines etc)
df_from_s3 = pd.read_csv(local_destination)
df_from_s3.describe(include='all')

,city,hotel_name,hotel_url,hotel_GPS,hotel_score,hotel_voters,hotel_description,feels_like,humidity,wind_speed,city_id,main_weather,main_weather_scores,final_score,lon,lat
count,1362,1362,1362,1362,1103,1179,1362,1362.000000,1362.000000,1362.000000,1362.000000,1362,1362.0,1362.000000,1362.000000,1362.000000
unique,5,1113,1362,1036,60,285,1120,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
top,Biarritz,CITY VIEW KEYWEEK Apartment in the heart of Bi...,https://www.booking.com/hotel/fr/la-maison-de-...,"42.52215500,3.08856800","9,0",1 expérience vécue,Doté d'une connexion Wi-Fi gratuite et offrant...,NaN,NaN,NaN,NaN,clear sky,NaN,NaN,NaN,NaN
freq,594,6,1,8,77,139,6,NaN,NaN,NaN,NaN,1362,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.753281,75.273605,2.667228,11.468429,NaN,5.0,632.801762,0.875136,43.293314
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.656682,5.544607,0.137270,7.710844,NaN,0.0,7.918601,2.674697,0.392103
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.368000,67.625000,2.464000,1.000000,NaN,5.0,616.000000,-1.559278,42.525050
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.368000,67.675000,2.579000,9.000000,NaN,5.0,627.000000,-1.559278,43.452277
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.520500,79.875000,2.579000,9.000000,NaN,5.0,636.000000,-1.473666,43.483252
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.374250,79.950000,2.830750,14.000000,NaN,5.0,636.000000,3.914252,43.483252


In [8]:
# Remove lines with missing info
df_from_s3=df_from_s3.dropna()
df_from_s3.info()
# Convert values in hotel_score and hotel_voters in float and integer
df_from_s3['hotel_score'] = df_from_s3['hotel_score'].str.replace(',', '.').astype(float)
df_from_s3['hotel_voters'] = df_from_s3['hotel_voters'].apply(lambda x: x.split()[0]).astype(int)
# Divide hotel_GPS into 2 columns, convert to float
df_from_s3[['hotel_lat', 'hotel_lon']] = df_from_s3['hotel_GPS'].str.split(',', expand=True).astype(float)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103 entries, 0 to 1361
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   city                 1103 non-null   object 
 1   hotel_name           1103 non-null   object 
 2   hotel_url            1103 non-null   object 
 3   hotel_GPS            1103 non-null   object 
 4   hotel_score          1103 non-null   object 
 5   hotel_voters         1103 non-null   object 
 6   hotel_description    1103 non-null   object 
 7   feels_like           1103 non-null   float64
 8   humidity             1103 non-null   float64
 9   wind_speed           1103 non-null   float64
 10  city_id              1103 non-null   float64
 11  main_weather         1103 non-null   object 
 12  main_weather_scores  1103 non-null   float64
 13  final_score          1103 non-null   float64
 14  lon                  1103 non-null   float64
 15  lat                  1103 non-null   f

In [9]:
df_from_s3.sort_values(by='hotel_score',ascending=False)
print("We notice that hotel_score of 10 has sometimes only 1 or 2 voters, which might not be sufficient for judgement.")
print("Number of hotels: ",len(df_from_s3))
to_keep = (df_from_s3['hotel_voters'] >= 5)
df_transformed = df_from_s3.loc[to_keep]
print("Number of hotels with more than 5 voters: ",len(df_transformed))

We notice that hotel_score of 10 has sometimes only 1 or 2 voters, which might not be sufficient for judgement.
Number of hotels:  1103
Number of hotels with more than 5 voters:  690


In [10]:
df_transformed.columns

Index(['city', 'hotel_name', 'hotel_url', 'hotel_GPS', 'hotel_score',
       'hotel_voters', 'hotel_description', 'feels_like', 'humidity',
       'wind_speed', 'city_id', 'main_weather', 'main_weather_scores',
       'final_score', 'lon', 'lat', 'hotel_lat', 'hotel_lon'],
      dtype='object')

In [11]:
# Define how should look the final table for managers

# Let's instanciate a declarative base to be able to use our python class
Base = declarative_base()

class Scraping(Base):
    # d'abord on doit donner un nom à notre table (obligé d'écrire tablename comme ca)
    __tablename__ = "Hotels"

    # Each parameter corresponds to a column in our DB table
    id = Column(Integer, primary_key=True, autoincrement=True)
    city = Column(String)
    hotel_name = Column(String)
    hotel_url = Column(String)
    hotel_score = Column(Float)
    hotel_voters = Column(Integer)
    hotel_description = Column(String)
    feels_like = Column(Float)
    humidity = Column(Float)
    wind_speed = Column(Float)
    city_id = Column(Float)
    main_weather = Column(String)
    main_weather_scores = Column(Float)
    final_score = Column(Float)
    lon = Column(Float)
    lat = Column(Float)
    hotel_lat = Column(Float)
    hotel_lon = Column(Float)

    def __repr__(self):
    # obligé de l'écrire comme ca exactement
        # return "<User(name='{}', fullname='{}', nickname='{}')>".format(self.name, self.fullname, self.nickname)
        columns = [column.name for column in Scraping.__table__.columns]
        values = [getattr(self, column) for column in columns]
        values_str = ', '.join([f'{column}={value}' for column, value in zip(columns, values)])
        return f"<Scraping({values_str})>"

### Load

In [18]:
# Create a SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}")

In [19]:
Base.metadata.create_all(engine)
# Creates the database schema based on our SQLAlchemy model definition (so the Base instance with the data 
# associated, like the table Hotels)

# In PGadmin, refresh then go to: project-getaround-db/Databases/postgres/Schema/public/Tables/Hotels
# Command QUERY TOOL to code in SQL

In [20]:
# Add all the data from the dataset
df_transformed.to_sql("Hotels", engine, if_exists="replace", index=False)

690

# Pick up best hotels

In [21]:
# Imagine now we are someone else from the company, querying the created SQL database to find out what are
# 20 best hotels for a 5 days trip in one of the 5top cities.

# The person would first create a new engine and a new session
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}")
Session = sessionmaker(bind=engine)
session = Session()

In [22]:
# Do we have comparable number of hotels for the 5 cities ?

query = """
SELECT city, COUNT(DISTINCT hotel_name) AS num_hotel_names
FROM public."Hotels"
GROUP BY city
"""

hotels_per_city = pd.read_sql(query, engine)
display(hotels_per_city)

print("There are quite similar number of hotels for each city.")

,city,num_hotel_names
0,Aigues Mortes,83
1,Bayonne,82
2,Biarritz,248
3,Collioure,95
4,Saintes Maries de la mer,87


There are quite similar number of hotels for each city.


In [23]:
# Find the 20 best hotels

query = """
SELECT city, hotel_name, hotel_score, hotel_voters, hotel_lat, hotel_lon
FROM public."Hotels"
ORDER BY hotel_score DESC
LIMIT 20
"""

# We need quotes for hotel_GPS because of the capital letters creating problem

top20_hotels = pd.read_sql(query, engine)
display(top20_hotels)

,city,hotel_name,hotel_score,hotel_voters,hotel_lat,hotel_lon
0,Biarritz,"Villa de charme parc fleuri, arboré et piscine...",10.0,5,43.464032,-1.547739
1,Saintes Maries de la mer,Résidence L’Oiseau des Sables,9.9,57,43.452052,4.431091
2,Aigues Mortes,Au Cœur des Remparts,9.9,145,43.565401,4.192973
3,Aigues Mortes,Marcelle en Camargue,9.9,25,43.566156,4.192345
4,Saintes Maries de la mer,"La Villa Vaihéré en Camargue, l'Hibiscus et le...",9.8,17,43.517299,4.374500
5,Biarritz,"Appartement Biarritz hypercentre calme, plage ...",9.8,5,43.481044,-1.558177
6,Aigues Mortes,La maison perchee,9.8,6,43.566500,4.189476
7,Aigues Mortes,Mas Sainte Marie,9.7,48,43.582241,4.247664
8,Aigues Mortes,Le Repaire du Soleil,9.7,10,43.565545,4.189661
9,Aigues Mortes,Séjour atypique et insolite sur notre péniche ...,9.7,60,43.581709,4.208536


In [24]:
fig = px.scatter_mapbox(
    top20_hotels, 
    title="Top 20 hotels for our kite surfing trip", 
    lat="hotel_lat", 
    lon="hotel_lon", 
    zoom=5, 
    mapbox_style="open-street-map",
    size="hotel_score",
    color = 'city',
    hover_name= 'hotel_name',
    hover_data = ['city', 'hotel_score','hotel_voters'],

)


fig.show()

In [25]:
# Close the session
session.close()